In [1]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
from build import TreeBuilding
from decision import haversine

%matplotlib inline

In [2]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [3]:
trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

In [4]:
tb = TreeBuilding()
tb = tb.build_tree(trajectories, targets, 1, 4000)

length 4328
('len', 4328)
length 3756
length 110


In [5]:
tb.root.prediction

[-8.6112507372920515, 41.18673345887246]

In [7]:
def predict_destination(traj_tree, trajectory):
    current_node = traj_tree.root
    
    for idx, point in enumerate(trajectory):
        if len(trajectory) > (idx + 1):
            if (current_node.left == None) and (current_node.right == None):
                prediction = current_node.prediction
                
            elif (current_node.left == None) and (current_node.right != None):
                current_node = current_node.right
                
            elif (current_node.left != None) and (current_node.right == None):
                current_node = current_node.left
                
            else:
                if haversine(point, current_node.decision_point) < current_node.radius:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
                    
        else:
            prediction = current_node.prediction
            
    return prediction

In [8]:
predict_destination(tb, trajectories[3])

[-8.6108782169274551, 41.186573344950212]

In [10]:
pred = []
pred.append(tb.root.prediction)
pred.append(tb.root.left.prediction)
pred.append(tb.root.right.prediction)

In [16]:
pred[0][1]

41.18673345887246

In [12]:
import gmplot

gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)

#gmap.plot(heat_lats, heat_lons, 'cornflowerblue', edge_width=10)
#gmap.scatter(heat_lats, heat_lons, '#3B0B39', size=10, marker=False)
gmap.scatter(pred[0][1], pred[0][0], '#006400', size=10, marker=False)
gmap.scatter(pred[1][1], pred[1][0], '#8B0000', size=10, marker=False)
gmap.scatter(pred[2][1], pred[2][0], '#000072', size=10, marker=False)

#gmap.heatmap(heat_lats, heat_lons)

gmap.draw("3predictions.html")

TypeError: zip argument #1 must support iteration